### Notebook for checking it the responses from the LLM are varied

In [35]:
import json
from pprint import pprint

In [36]:
with open ('responses.json') as f:
    responses = json.load(f)

In [37]:
location_time_list = {}
for i in range(len(responses)):
    json_response = json.loads(responses[i])
    location_time_list[i] = json_response

pprint(location_time_list)

{0: {'10': {'activity': 'study', 'location': 'IB'},
     '11': {'activity': 'study', 'location': 'IB'},
     '12': {'activity': 'lunch', 'location': 'restaurant'},
     '13': {'activity': 'lunch', 'location': 'restaurant'},
     '14': {'activity': 'study', 'location': 'IG'},
     '15': {'activity': 'study', 'location': 'IG'},
     '16': {'activity': 'study', 'location': 'IG'},
     '17': {'activity': 'leisure', 'location': 'restaurant'},
     '18': {'activity': 'dinner', 'location': 'restaurant'},
     '19': {'activity': 'shopping', 'location': 'store'},
     '20': {'activity': 'sports', 'location': 'gym'},
     '21': {'activity': 'shopping', 'location': 'store'},
     '22': {'activity': 'relaxation', 'location': 'home'},
     '23': {'activity': 'rest', 'location': 'home'},
     '7': {'activity': 'wake up', 'location': 'home'},
     '8': {'activity': 'study', 'location': 'IB'},
     '9': {'activity': 'study', 'location': 'IB'}},
 1: {'10': {'activity': 'study', 'location': 'IB'},
     

In [40]:
times = {}
locations = []
for i in range(len(location_time_list[0])):
    
    times[i + 7] = {}
    for j in range(len(location_time_list)):
        local = location_time_list[j][f'{i + 7}']['location']
        if local not in locations:
            locations.append(local)
        try:
            times[i + 7][f'{local}'] += 1 
        except KeyError:
            times[i + 7][f'{local}'] = 1

    for key in times[i + 7]:
        times[i + 7][key] /= len(location_time_list) / 100

pprint(locations)

['home', 'IB', 'restaurant', 'IG', 'supermarket', 'store', 'gym', 'cafe']


In [15]:
mermaid_code = "graph TD\n"
for key, values in times.items():
    for val, prob in values.items():
        if key == 7:
            # Starting key node
            mermaid_code += f"{key}((7)) --> {key}_{val}({val})\n"
        else:
            for prev_val in times[key - 1]:
                mermaid_code += f"{key - 1}_{prev_val} --> |{prob}%| {key}_{val}({val})\n"

with open ("mermaid_code.txt", "w") as f:
    f.write(mermaid_code)
print(mermaid_code)

graph TD
7((7)) --> 7_home(home)
7_home --> |100.0%| 8_IB(IB)
8_IB --> |60.0%| 9_IB(IB)
8_IB --> |20.0%| 9_home(home)
8_IB --> |20.0%| 9_cafe(cafe)
9_IB --> |100.0%| 10_IB(IB)
9_home --> |100.0%| 10_IB(IB)
9_cafe --> |100.0%| 10_IB(IB)
10_IB --> |80.0%| 11_IB(IB)
10_IB --> |20.0%| 11_home(home)
11_IB --> |20.0%| 12_IB(IB)
11_home --> |20.0%| 12_IB(IB)
11_IB --> |80.0%| 12_restaurant(restaurant)
11_home --> |80.0%| 12_restaurant(restaurant)
12_IB --> |20.0%| 13_restaurant(restaurant)
12_restaurant --> |20.0%| 13_restaurant(restaurant)
12_IB --> |60.0%| 13_IB(IB)
12_restaurant --> |60.0%| 13_IB(IB)
12_IB --> |20.0%| 13_supermarket(supermarket)
12_restaurant --> |20.0%| 13_supermarket(supermarket)
13_restaurant --> |20.0%| 14_supermarket(supermarket)
13_IB --> |20.0%| 14_supermarket(supermarket)
13_supermarket --> |20.0%| 14_supermarket(supermarket)
13_restaurant --> |80.0%| 14_IB(IB)
13_IB --> |80.0%| 14_IB(IB)
13_supermarket --> |80.0%| 14_IB(IB)
14_supermarket --> |80.0%| 15_IB(IB)
14_

![
](attachment:image.png)